# Задача №0

Дорешайте задачу про риды и BWT из файла practice4.ipynb. В ответе помимо кода укажите количество ридов первой, второй и третьей категории, а также вкратце опишите ход решения.

Это домашнее задание можно выполнять целиком в этом ноутбуке, либо алгоритмы написать в отдельном файле и импортировать сюда, для использования. В папке data лежат два файла islands.fasta и nonIslands.fasta. В них хранятся прочтения из CpG островков и из обычных участков генома соответственно, этими данными нужно будет воспользоваться в первом задании.

In [1]:
from pysuffixarray.core import SuffixArray
from Bio import SeqIO

In [2]:
class BTW_searcher():
    def __init__(self, sequence):

        self.suffix_array = SuffixArray(sequence)

        self.bwt_text = ''.join(sequence[i-1] if i != 0 else '$' for i in self.suffix_array.suffix_array())

        self.occ = {letter: [0 for _ in range(len(self.bwt_text))] for letter in '$ACGT'}

        for i, letter in enumerate(self.bwt_text):
            for ch in 'ACGT':
                if i != 0:
                    self.occ[ch][i] = self.occ[ch][i-1]
            if letter in self.occ.keys():
                self.occ[letter][i] += 1
    
        self.count = {}
        total = 0
        for letter in '$ACGT':
            self.count[letter] = total
            total += self.bwt_text.count(letter)

    def bwt_pattern_search(self, pattern):
        top = 0
        bottom = len(self.bwt_text) - 1
        for letter in reversed(pattern):
            if letter in self.occ:
                top = self.count[letter] + (0 if top == 0 else self.occ[letter][top-1])
                bottom = self.count[letter] + self.occ[letter][bottom] - 1
            else:
                return []
            if top > bottom:
                return []
        return self.suffix_array.suffix_array()[top:bottom+1]
    

def get_btw(seq):
    sa = SuffixArray(seq) 
    return ''.join(seq[i-1] if i != 0 else '$' for i in sa.suffix_array())


In [3]:
seqs = SeqIO.parse(r"/home/pk/Desktop/BioinformaticsCourse2024PK/homework/1_4/BWT_folder/genome.fa", "fasta")

for seq in seqs:
    sequence = str(seq.seq)

btw_text = get_btw(sequence)
bwts = BTW_searcher(sequence)

Риды без мутаций найти просто - нужно просто найти паттерн с помощью реализованного метода bwt_pattern_search. Если массив индексов непустой - вхождение подстроки есть

In [4]:
count = 0
with open(r"/home/pk/Desktop/BioinformaticsCourse2024PK/homework/1_4/BWT_folder/sample_reads.fasta", "r") as file:
    for record in SeqIO.parse(file, "fasta"):
        read_sequence = str(record.seq)
        pattern_insert = bwts.bwt_pattern_search(read_sequence)
        if len(pattern_insert) > 0:
            count += 1
            #  если надо вывести конкретные риды:
            #print("Processing read:", read_sequence)
            #print(*pattern_insert)
print('Число ридов без мутаций -', count)

Число ридов без мутаций - 45


Чтобы найти риды с несколькими мутациями (мы знаем их число - m), нужно разделить строку паттерн на (m+1) частей, и проверить, какая из них входит в последовательность (одна часть подстроки точно будет входить без ошибок в нашу строку). Если нашли вхождение - проверяем посимвольно соответствующие подстроку и часть строки. Если число несовпадающих символов равно числу m, то мы нашли подстроку в строке с заданным числом мутаций

In [58]:
def mutation_search(sequence, subsequence, num_mutations):
    i = 0
    k = len(subsequence)//(num_mutations+1)
    segments = []
    for _ in range(num_mutations):
        segments.append(subsequence[i:i+k])
        i += k
    segments.append(subsequence[i:])
    for segment_idx, segment in enumerate(segments):
            idxs = bwts.bwt_pattern_search(segment)
            for idx in idxs:
                start = idx-segment_idx*k
                end = start + len(subsequence)
                m = 0
                for char_idx in range(start, end):
                    if sequence[char_idx] != subsequence[char_idx-start]:
                        m += 1
                if m == num_mutations:
                    #  если надо вывести конкретные риды:
                    # print("Processing read:", subsequence)
                    # print(*pattern_insert)
                    return True
    return False

count_0 = 0
count_1 = 0
count_5 = 0
with open(r"/home/pk/Desktop/BioinformaticsCourse2024PK/homework/1_4/BWT_folder/sample_reads.fasta", "r") as file:
    for record in SeqIO.parse(file, "fasta"):
        subsequence = str(record.seq)
        if mutation_search(sequence, subsequence, 0):
            count_0 += 1
        if mutation_search(sequence, subsequence, 1):
            count_1 += 1
        if mutation_search(sequence, subsequence, 5):
            count_5 += 1

print('Число паттернов без мутаций -', count_0)
print('Число паттернов с одной мутацией -', count_1)
print('Число паттернов с пятью мутациями -', count_5)
print(count_0+count_1+count_5)

Число паттернов без мутаций - 45
Число паттернов с одной мутацией - 31
Число паттернов с пятью мутациями - 24
100
